In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import os
path = '/content/drive/My Drive/kaggle/twitter/'
os.chdir(path) # カレントディレクトリを指定
os.getcwd()

'/content/drive/My Drive/kaggle/twitter'

In [ ]:
!pip install kaggle

In [ ]:
!mkdir -p /root/.kaggle/
!cp /content/drive/"My Drive"/kaggle/config/kaggle.json /root/.kaggle/kaggle.json

In [ ]:
import os
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()
# competitions = api.competitions_list(category="featured")
# for comp in competitions:
#     print(comp.ref,comp.reward,comp.userRank,sep=',')

In [ ]:
competitions = 'tweet-sentiment-extraction'


for json_dict in api.competitions_data_list_files('tweet-sentiment-extraction'):
  api.competition_download_file(competitions, json_dict['name'], './data/')

100%|██████████| 307k/307k [00:00<00:00, 23.1MB/s]


100%|██████████| 41.4k/41.4k [00:00<00:00, 4.43MB/s]


100%|██████████| 1.23M/1.23M [00:00<00:00, 61.1MB/s]

In [ ]:
!kaggle datasets download -d abhishek/tweet-train-folds -p ./data/
!kaggle datasets download -d abhishek/roberta-base -p ./model/
!kaggle datasets download -d thanatoz/tweetsentiment-pseudo-labelled -p ./data/

tweet-train-folds.zip: Skipping, found more recently modified local copy (use --force to force download)
roberta-base.zip: Skipping, found more recently modified local copy (use --force to force download)
  0% 0.00/323k [00:00<?, ?B/s]
100% 323k/323k [00:00<00:00, 20.6MB/s]


In [ ]:
import glob
import zipfile

def unzip(path):
  zip_files = glob.glob(f"{path}/**/*.zip")
  for file in zip_files:
    with zipfile.ZipFile(file) as existing_zip:
        existing_zip.extractall(path=os.path.dirname(file))

unzip(path)

In [ ]:
!pip install transformers
!pip install tokenizers

     |████████████████████████████████| 675kB 2.8MB/s 
     |████████████████████████████████| 1.1MB 13.5MB/s 
     |████████████████████████████████| 890kB 14.8MB/s 
     |████████████████████████████████| 3.8MB 23.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=613fea8a33d5b2b9158fc88292bce9a5fbcac88ed7a6986410e4cf3aa16a911b
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


# Code

In [ ]:
use_colab = True # not kaggle kernel

In [ ]:
import numpy as np
import torch


class utils:

    class AverageMeter:
        """
        Computes and stores the average and current value
        """
        def __init__(self):
            self.reset()

        def reset(self):
            self.val = 0
            self.avg = 0
            self.sum = 0
            self.count = 0

        def update(self, val, n=1):
            self.val = val
            self.sum += val * n
            self.count += n
            self.avg = self.sum / self.count


    class EarlyStopping:
        def __init__(self, patience=7, mode="max", delta=0.001):
            self.patience = patience
            self.counter = 0
            self.mode = mode
            self.best_score = None
            self.early_stop = False
            self.delta = delta
            if self.mode == "min":
                self.val_score = np.Inf
            else:
                self.val_score = -np.Inf

        def __call__(self, epoch_score, model, model_path):

            if self.mode == "min":
                score = -1.0 * epoch_score
            else:
                score = np.copy(epoch_score)

            if self.best_score is None:
                self.best_score = score
                self.save_checkpoint(epoch_score, model, model_path)
            elif score < self.best_score + self.delta:
                self.counter += 1
                print('EarlyStopping counter: {} out of {}'.format(self.counter, self.patience))
                if self.counter >= self.patience:
                    self.early_stop = True
            else:
                self.best_score = score
                self.save_checkpoint(epoch_score, model, model_path)
                self.counter = 0

        def save_checkpoint(self, epoch_score, model, model_path):
            if epoch_score not in [-np.inf, np.inf, -np.nan, np.nan]:
                print('Validation score improved ({} --> {}). Saving model!'.format(self.val_score, epoch_score))
                torch.save(model.state_dict(), model_path)
            self.val_score = epoch_score


    def jaccard(str1, str2): 
        a = set(str1.lower().split()) 
        b = set(str2.lower().split())
        c = a.intersection(b)
        return float(len(c)) / (len(a) + len(b) - len(c))

In [ ]:
import os
import re
import torch
import pandas as pd
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from torch.optim import lr_scheduler

from sklearn import model_selection
from sklearn.model_selection import StratifiedKFold, GroupKFold
from sklearn import metrics
import transformers
import tokenizers
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
from tqdm.autonotebook import tqdm

if not use_colab:
  import utils

In [ ]:
ROBERTA_PATH = "../input/roberta-base" if not use_colab else './model/'
TOKENIZER = tokenizers.ByteLevelBPETokenizer(
    vocab_file=f"{ROBERTA_PATH}/vocab.json", 
    merges_file=f"{ROBERTA_PATH}/merges.txt", 
    lowercase=True,
    add_prefix_space=True
)
DATA_PATH = "../input/tweet-train-folds/" if not use_colab else './data/'
TRAINING_FILE = os.path.join(DATA_PATH, 'train_folds.csv')
MAX_LEN = 128
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 8
EPOCHS = 3

In [ ]:
def process_data(tweet, selected_text, sentiment, tokenizer, max_len):
    """
    Processes the tweet and outputs the features necessary for model training and inference.
    
    Note: there are some differences between this and the RoERTa version (roberta-base), mostly due to differences in token codes and special tokens
    """
    tweet = " " + " ".join(str(tweet).split())
    selected_text = " " + " ".join(str(selected_text).split())

    len_st = len(selected_text) - 1
    idx0 = None
    idx1 = None

    for ind in (i for i, e in enumerate(tweet) if e == selected_text[1]):
        if " " + tweet[ind: ind+len_st] == selected_text:
            idx0 = ind
            idx1 = ind + len_st - 1
            break

    char_targets = [0] * len(tweet)
    if idx0 != None and idx1 != None:
        for ct in range(idx0, idx1 + 1):
            char_targets[ct] = 1
    
    tok_tweet = tokenizer.encode(tweet)
    input_ids_orig = tok_tweet.ids
    tweet_offsets = tok_tweet.offsets
    
    target_idx = []
    for j, (offset1, offset2) in enumerate(tweet_offsets):
        if sum(char_targets[offset1: offset2]) > 0:
            target_idx.append(j)
    
    targets_start = target_idx[0]
    targets_end = target_idx[-1]

    sentiment_id = {
        'positive': 1313,
        'negative': 2430,
        'neutral': 7974
    }
    
    input_ids = [0] + [sentiment_id[sentiment]] + [2] + [2] + input_ids_orig + [2]
    token_type_ids = [0, 0, 0, 0] + [0] * (len(input_ids_orig) + 1)
    mask = [1] * len(token_type_ids)
    tweet_offsets = [(0, 0)] * 4 + tweet_offsets + [(0, 0)]
    targets_start += 4
    targets_end += 4

    padding_length = max_len - len(input_ids)
    if padding_length > 0:
        input_ids = input_ids + ([1] * padding_length)
        mask = mask + ([0] * padding_length)
        token_type_ids = token_type_ids + ([0] * padding_length)
        tweet_offsets = tweet_offsets + ([(0, 0)] * padding_length)
    
    return {
        'ids': input_ids,
        'mask': mask,
        'token_type_ids': token_type_ids,
        'targets_start': targets_start,
        'targets_end': targets_end,
        'orig_tweet': tweet,
        'orig_selected': selected_text,
        'sentiment': sentiment,
        'offsets': tweet_offsets
    }

In [ ]:
tweet = 'Sooo SAD I will miss you here in San Diego!!!'
selected_text = 'Sooo SAD'
sentiment = 'negative'

# MAX_LEN=128
data = process_data(tweet, selected_text, sentiment, TOKENIZER, MAX_LEN)

In [ ]:
print(data.keys())

dict_keys(['ids', 'mask', 'token_type_ids', 'targets_start', 'targets_end', 'orig_tweet', 'orig_selected', 'sentiment', 'offsets'])


In [ ]:
print(data['ids'])

[0, 2430, 2, 2, 98, 3036, 5074, 939, 40, 2649, 47, 259, 11, 15610, 1597, 2977, 16506, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [ ]:
print(data['orig_selected'])

 Sooo SAD


In [ ]:
tweet = 'Sooo SAD I will miss you here in San Diego!!!'
selected_text = 'Sooo SAD'

[0] + 'negative' + [2] + [2] + text + [2]


In [ ]:
print(data['targets_end']) # ０〜３番目はcls_token, sentiment, sep_token, sep_token

6


In [ ]:
print(data['ids'])

[0, 2430, 2, 2, 98, 3036, 5074, 939, 40, 2649, 47, 259, 11, 15610, 1597, 2977, 16506, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [ ]:
print(data['offsets'])

[(0, 0), (0, 0), (0, 0), (0, 0), (0, 3), (3, 5), (5, 9), (9, 11), (11, 16), (16, 21), (21, 25), (25, 30), (30, 33), (33, 37), (37, 41), (41, 43), (43, 46), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 

In [ ]:
TOKENIZER.id_to_token(5074)

'Ġsad'

In [ ]:
class TweetDataset:
    """
    Dataset which stores the tweets and returns them as processed features
    """
    def __init__(self, tweet, sentiment, selected_text):
        self.tweet = tweet
        self.sentiment = sentiment
        self.selected_text = selected_text
        self.tokenizer = TOKENIZER
        self.max_len = MAX_LEN
    
    def __len__(self):
        return len(self.tweet)

    def __getitem__(self, item):
        data = process_data(
            self.tweet[item], 
            self.selected_text[item], 
            self.sentiment[item],
            self.tokenizer,
            self.max_len
        )

        # Return the processed data where the lists are converted to `torch.tensor`s
        return {
            'ids': torch.tensor(data["ids"], dtype=torch.long),
            'mask': torch.tensor(data["mask"], dtype=torch.long),
            'token_type_ids': torch.tensor(data["token_type_ids"], dtype=torch.long),
            'targets_start': torch.tensor(data["targets_start"], dtype=torch.long),
            'targets_end': torch.tensor(data["targets_end"], dtype=torch.long),
            'orig_tweet': data["orig_tweet"],
            'orig_selected': data["orig_selected"],
            'sentiment': data["sentiment"],
            'offsets': torch.tensor(data["offsets"], dtype=torch.long)
        }

In [ ]:
class TweetModel(transformers.BertPreTrainedModel):
    """
    Model class that combines a pretrained bert model with a linear later
    """
    def __init__(self, conf):
        super(TweetModel, self).__init__(conf)
        # Load the pretrained RobBERTa model
        self.roberta = transformers.RobertaModel.from_pretrained(ROBERTA_PATH, config=conf)
        # Set 10% dropout to be applied to the RobBERTa backbone's output
        self.drop_out = nn.Dropout(0.1)
        # 768 is the dimensionality of roberta-base's hidden representations
        # Multiplied by 2 since the forward pass concatenates the last two hidden representation layers
        # The output will have two dimensions ("start_logits", and "end_logits")
        self.l0 = nn.Linear(768 * 2, 2)
        torch.nn.init.normal_(self.l0.weight, std=0.02)
    
    def forward(self, ids, mask, token_type_ids):
        # Return the hidden states from the BERT backbone
        _, _, out = self.roberta(
            ids,
            attention_mask=mask,
            token_type_ids=token_type_ids
        ) # bert_layers x bs x SL x (768 * 2)

        # Concatenate the last two hidden states
        # This is done since experiments have shown that just getting the last layer
        # gives out vectors that may be too taylored to the original RoBERTa training objectives (MLM + NSP)
        # Sample explanation: https://bert-as-service.readthedocs.io/en/latest/section/faq.html#why-not-the-last-hidden-layer-why-second-to-last
        out = torch.cat((out[-1], out[-2]), dim=-1) # bs x SL x (768 * 2)
        # Apply 10% dropout to the last 2 hidden states
        out = self.drop_out(out) # bs x SL x (768 * 2)
        # The "dropped out" hidden vectors are now fed into the linear layer to output two scores
        logits = self.l0(out) # bs x SL x 2

        # Splits the tensor into start_logits and end_logits
        # (bs x SL x 2) -> (bs x SL x 1), (bs x SL x 1)
        start_logits, end_logits = logits.split(1, dim=-1)

        start_logits = start_logits.squeeze(-1) # (bs x SL)
        end_logits = end_logits.squeeze(-1) # (bs x SL)

        return start_logits, end_logits

In [ ]:
class TweetModel(transformers.BertPreTrainedModel):
    """
    Model class that combines a pretrained bert model with a linear later
    """
    def __init__(self, conf):
        super(TweetModel, self).__init__(conf)
        # Load the pretrained RobBERTa model
        self.roberta = transformers.RobertaModel.from_pretrained(ROBERTA_PATH, config=conf)
        # Set 10% dropout to be applied to the RobBERTa backbone's output
        self.drop_out = nn.Dropout(0.1)
        self.drop_out2 = nn.Dropout(0.1)
        # 768 is the dimensionality of roberta-base's hidden representations
        # Multiplied by 2 since the forward pass concatenates the last two hidden representation layers
        # The output will have two dimensions ("start_logits", and "end_logits")
        self.l0 = nn.Linear(768 * 2, 768)
        self.l1 = nn.Linear(768, 2)
        torch.nn.init.normal_(self.l0.weight, std=0.02)
        torch.nn.init.normal_(self.l1.weight, std=0.02)
    
    def forward(self, ids, mask, token_type_ids):
        # Return the hidden states from the BERT backbone
        _, _, out = self.roberta(
            ids,
            attention_mask=mask,
            token_type_ids=token_type_ids
        ) # bert_layers x bs x SL x (768 * 2)

        # Concatenate the last two hidden states
        # This is done since experiments have shown that just getting the last layer
        # gives out vectors that may be too taylored to the original RoBERTa training objectives (MLM + NSP)
        # Sample explanation: https://bert-as-service.readthedocs.io/en/latest/section/faq.html#why-not-the-last-hidden-layer-why-second-to-last
        out = torch.cat((out[-1], out[-2]), dim=-1) # bs x SL x (768 * 2)
        # Apply 10% dropout to the last 2 hidden states
        out = self.drop_out(out) # bs x SL x (768 * 2)
        # The "dropped out" hidden vectors are now fed into the linear layer to output two scores
        logits = self.l0(out) # bs x SL x 2
        out = self.drop_out2(out)
        logits = self.l1(logits)

        # Splits the tensor into start_logits and end_logits
        # (bs x SL x 2) -> (bs x SL x 1), (bs x SL x 1)
        start_logits, end_logits = logits.split(1, dim=-1)

        start_logits = start_logits.squeeze(-1) # (bs x SL)
        end_logits = end_logits.squeeze(-1) # (bs x SL)

        return start_logits, end_logits

In [ ]:
# class TweetModel(transformers.BertPreTrainedModel):
#     """
#     Model class that combines a pretrained bert model with a linear later
#     """
#     def __init__(self, conf):
#         super(TweetModel, self).__init__(conf)
#         # Load the pretrained RobBERTa model
#         self.roberta = transformers.RobertaModel.from_pretrained(ROBERTA_PATH, config=conf)
#         # Set 10% dropout to be applied to the RobBERTa backbone's output
#         self.drop_out = nn.Dropout(0.3)
#         # 768 is the dimensionality of roberta-base's hidden representations
#         # Multiplied by 2 since the forward pass concatenates the last two hidden representation layers
#         # The output will have two dimensions ("start_logits", and "end_logits")
#         self.l0 = nn.Linear(768 * 2, 2)
#         self.rnn = nn.RNN(input_size=2, hidden_size=2, num_layers=1)

#         torch.nn.init.normal_(self.l0.weight, std=0.02)
    
#     def forward(self, ids, mask, token_type_ids):
#         # Return the hidden states from the BERT backbone
#         _, _, out = self.roberta(
#             ids,
#             attention_mask=mask,
#             token_type_ids=token_type_ids
#         ) # bert_layers x bs x SL x (768 * 2)

#         # Concatenate the last two hidden states
#         # This is done since experiments have shown that just getting the last layer
#         # gives out vectors that may be too taylored to the original RoBERTa training objectives (MLM + NSP)
#         # Sample explanation: https://bert-as-service.readthedocs.io/en/latest/section/faq.html#why-not-the-last-hidden-layer-why-second-to-last
#         out = torch.cat((out[-1], out[-2]), dim=-1) # bs x SL x (768 * 2)
#         # Apply 10% dropout to the last 2 hidden states
#         out = self.drop_out(out) # bs x SL x (768 * 2)
#         # The "dropped out" hidden vectors are now fed into the linear layer to output two scores
#         logits = self.l0(out) # bs x SL x 2
#         logits, hn = self.rnn(logits)
#         # logits = torch.transpose(logits, 1, 2)

#         # Splits the tensor into start_logits and end_logits
#         # (bs x SL x 2) -> (bs x SL x 1), (bs x SL x 1)
#         start_logits, end_logits = logits.split(1, dim=-1)

#         start_logits = start_logits.squeeze(-1) # (bs x SL)
#         end_logits = end_logits.squeeze(-1) # (bs x SL)

#         return start_logits, end_logits

In [ ]:
# class TweetModel(transformers.BertPreTrainedModel):
#     """
#     Model class that combines a pretrained bert model with a linear later
#     """
#     def __init__(self, conf):
#         super(TweetModel, self).__init__(conf)
#         # Load the pretrained RobBERTa model
#         self.roberta = transformers.RobertaModel.from_pretrained(ROBERTA_PATH, config=conf)

#         self.drop_out_s = nn.Dropout(0.1)
#         self.conv_s1 = torch.nn.Conv1d(1536, 786, kernel_size=3, stride=1, padding=1)
#         self.leaky_relu_s1 = torch.nn.LeakyReLU()
#         self.conv_s2 = torch.nn.Conv1d(786, 128, kernel_size=3, stride=1, padding=1)
#         self.leaky_relu_s2 = torch.nn.LeakyReLU()
#         self.conv_s3 = torch.nn.Conv1d(128, 64, kernel_size=3, stride=1, padding=1)
#         self.ln_s = nn.Linear(64, 1)

#         self.drop_out_e = nn.Dropout(0.1)
#         self.conv_e1 = torch.nn.Conv1d(1536, 786, kernel_size=3, stride=1, padding=1)
#         self.leaky_relu_e1 = torch.nn.LeakyReLU()
#         self.conv_e2 = torch.nn.Conv1d(786, 128, kernel_size=3, stride=1, padding=1)
#         self.leaky_relu_e2 = torch.nn.LeakyReLU()
#         self.conv_e3 = torch.nn.Conv1d(128, 64, kernel_size=3, stride=1, padding=1)
#         self.ln_e = nn.Linear(64, 1)
    
#     def forward(self, ids, mask, token_type_ids):
#         # Return the hidden states from the BERT backbone
#         _, _, out = self.roberta(
#             ids,
#             attention_mask=mask,
#             token_type_ids=token_type_ids
#         )

#         out = torch.cat((out[-1], out[-2]), dim=-1) # bs x SL x (768 * 2)
#         out = torch.transpose(out, 1, 2) # [32, 192, 1536, (1)] -> [32, 1536, 192, (1)]

#         s = self.drop_out_s(out)
#         s = self.conv_s1(s)
#         s = self.leaky_relu_s1(s)
#         s = self.conv_s2(s)
#         s = self.leaky_relu_s2(s)
#         s = self.conv_s3(s)
#         s = torch.transpose(s, 1, 2)
#         s = self.ln_s(s)

#         e = self.drop_out_e(out)
#         e = self.conv_e1(e)
#         e = self.leaky_relu_e1(e)
#         e = self.conv_e2(e)
#         e = self.leaky_relu_e2(e)
#         e = self.conv_e3(e)
#         e = torch.transpose(e, 1, 2)
#         e = self.ln_e(e)
        

#         start_logits = s.squeeze(-1) # (bs x SL)
#         end_logits = e.squeeze(-1) # (bs x SL)

#         return start_logits, end_logits

In [ ]:
def loss_fn(start_logits, end_logits, start_positions, end_positions):
    """
    Return the sum of the cross entropy losses for both the start and end logits
    """
    loss_fct = nn.CrossEntropyLoss()
    start_loss = loss_fct(start_logits, start_positions)
    end_loss = loss_fct(end_logits, end_positions)
    total_loss = (start_loss + end_loss)
    return total_loss

In [ ]:
def train_fn(data_loader, model, optimizer, device, scheduler=None):
    """
    Trains the bert model on the twitter data
    """
    # Set model to training mode (dropout + sampled batch norm is activated)
    model.train()
    losses = utils.AverageMeter()
    jaccards = utils.AverageMeter()

    # Set tqdm to add loading screen and set the length
    tk0 = tqdm(data_loader, total=len(data_loader))
    
    # Train the model on each batch
    for bi, d in enumerate(tk0):

        ids = d["ids"]
        token_type_ids = d["token_type_ids"]
        mask = d["mask"]
        targets_start = d["targets_start"]
        targets_end = d["targets_end"]
        sentiment = d["sentiment"]
        orig_selected = d["orig_selected"]
        orig_tweet = d["orig_tweet"]
        targets_start = d["targets_start"]
        targets_end = d["targets_end"]
        offsets = d["offsets"]
        

        # Move ids, masks, and targets to gpu while setting as torch.long
        ids = ids.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        targets_start = targets_start.to(device, dtype=torch.long)
        targets_end = targets_end.to(device, dtype=torch.long)

        # Reset gradients
        model.zero_grad()
        # Use ids, masks, and token types as input to the model
        # Predict logits for each of the input tokens for each batch
        outputs_start, outputs_end = model(
            ids=ids,
            mask=mask,
            token_type_ids=token_type_ids,
        ) # (bs x SL), (bs x SL)
        # Calculate batch loss based on CrossEntropy
        loss = loss_fn(outputs_start, outputs_end, targets_start, targets_end)
        # Calculate gradients based on loss
        loss.backward()
        # Adjust weights based on calculated gradients
        optimizer.step()
        # Update scheduler
        scheduler.step()
        
        # Apply softmax to the start and end logits
        # This squeezes each of the logits in a sequence to a value between 0 and 1, while ensuring that they sum to 1
        # This is similar to the characteristics of "probabilities"
        outputs_start = torch.softmax(outputs_start, dim=1).cpu().detach().numpy()
        outputs_end = torch.softmax(outputs_end, dim=1).cpu().detach().numpy()
        
        # Calculate the jaccard score based on the predictions for this batch
        jaccard_scores = []
        for px, tweet in enumerate(orig_tweet):
            selected_tweet = orig_selected[px]
            tweet_sentiment = sentiment[px]
            jaccard_score, _ = calculate_jaccard_score(
                original_tweet=tweet, # Full text of the px'th tweet in the batch
                target_string=selected_tweet, # Span containing the specified sentiment for the px'th tweet in the batch
                sentiment_val=tweet_sentiment, # Sentiment of the px'th tweet in the batch
                idx_start=np.argmax(outputs_start[px, :]), # Predicted start index for the px'th tweet in the batch
                idx_end=np.argmax(outputs_end[px, :]), # Predicted end index for the px'th tweet in the batch
                offsets=offsets[px] # Offsets for each of the tokens for the px'th tweet in the batch
            )
            jaccard_scores.append(jaccard_score)
        # Update the jaccard score and loss
        # For details, refer to `AverageMeter` in https://www.kaggle.com/abhishek/utils
        jaccards.update(np.mean(jaccard_scores), ids.size(0))
        losses.update(loss.item(), ids.size(0))
        # Print the average loss and jaccard score at the end of each batch
        tk0.set_postfix(loss=losses.avg, jaccard=jaccards.avg)

In [ ]:
import re

def calculate_jaccard_score(
    original_tweet, 
    target_string, 
    sentiment_val, 
    idx_start, 
    idx_end, 
    offsets,
    verbose=False):
    """
    Calculate the jaccard score from the predicted span and the actual span for a batch of tweets
    """
    
    # A span's start index has to be greater than or equal to the end index
    # If this doesn't hold, the start index is set to equal the end index (the span is a single token)
    if idx_end < idx_start:
        idx_end = idx_start
    
    # Combine into a string the tokens that belong to the predicted span
    filtered_output  = ""
    for ix in range(idx_start, idx_end + 1):
        filtered_output += original_tweet[offsets[ix][0]: offsets[ix][1]]
        # If the token is not the last token in the tweet, and the ending offset of the current token is less
        # than the beginning offset of the following token, add a space.
        # Basically, add a space when the next token (word piece) corresponds to a new word
        if (ix+1) < len(offsets) and offsets[ix][1] < offsets[ix+1][0]:
            filtered_output += " "

            
    if sentiment_val == "neutral":
        # neutralのtweetはURLを除いた全文を回答にしている
        http_search = re.compile(r'(https?://[a-zA-Z0-9.-/]*)')
        match = http_search.search(original_tweet)
        if match:
            replace = match.group(1).split('://')
            original_tweet = original_tweet.replace(match.group(1), '')
        filtered_output = original_tweet
    elif len(original_tweet.split()) < 2:
        # １単語のtweetはそのまま採用
        filtered_output = original_tweet
    # Set the predicted output as the original tweet when the tweet's sentiment is "neutral", or the tweet only contains one word
    # if sentiment_val == "neutral" or len(original_tweet.split()) < 2:
    #     filtered_output = original_tweet

    # Calculate the jaccard score between the predicted span, and the actual span
    # The IOU (intersection over union) approach is detailed in the utils module's `jaccard` function:
    # https://www.kaggle.com/abhishek/utils
    jac = utils.jaccard(target_string.strip(), filtered_output.strip())
    return jac, filtered_output

In [ ]:
def eval_fn(data_loader, model, device):
    """
    Evaluation function to predict on the test set
    """
    # Set model to evaluation mode
    # I.e., turn off dropout and set batchnorm to use overall mean and variance (from training), rather than batch level mean and variance
    # Reference: https://github.com/pytorch/pytorch/issues/5406
    model.eval()
    losses = utils.AverageMeter()
    jaccards = utils.AverageMeter()
    
    # Turns off gradient calculations (https://datascience.stackexchange.com/questions/32651/what-is-the-use-of-torch-no-grad-in-pytorch)
    with torch.no_grad():
        tk0 = tqdm(data_loader, total=len(data_loader))
        # Make predictions and calculate loss / jaccard score for each batch
        for bi, d in enumerate(tk0):
            ids = d["ids"]
            token_type_ids = d["token_type_ids"]
            mask = d["mask"]
            sentiment = d["sentiment"]
            orig_selected = d["orig_selected"]
            orig_tweet = d["orig_tweet"]
            targets_start = d["targets_start"]
            targets_end = d["targets_end"]
            offsets = d["offsets"].numpy()
            
            # Move ids, masks, and targets to gpu while setting as torch.long
            ids = ids.to(device, dtype=torch.long)
            token_type_ids = token_type_ids.to(device, dtype=torch.long)
            mask = mask.to(device, dtype=torch.long)
            targets_start = targets_start.to(device, dtype=torch.long)
            targets_end = targets_end.to(device, dtype=torch.long)

            # Move tensors to GPU for faster matrix calculations
            ids = ids.to(device, dtype=torch.long)
            token_type_ids = token_type_ids.to(device, dtype=torch.long)
            mask = mask.to(device, dtype=torch.long)
            targets_start = targets_start.to(device, dtype=torch.long)
            targets_end = targets_end.to(device, dtype=torch.long)

            # Predict logits for start and end indexes
            outputs_start, outputs_end = model(
                ids=ids,
                mask=mask,
                token_type_ids=token_type_ids
            )
            # Calculate loss for the batch
            loss = loss_fn(outputs_start, outputs_end, targets_start, targets_end)
            # Apply softmax to the predicted logits for the start and end indexes
            # This converts the "logits" to "probability-like" scores
            outputs_start = torch.softmax(outputs_start, dim=1).cpu().detach().numpy()
            outputs_end = torch.softmax(outputs_end, dim=1).cpu().detach().numpy()
            # Calculate jaccard scores for each tweet in the batch
            jaccard_scores = []
            for px, tweet in enumerate(orig_tweet):
                selected_tweet = orig_selected[px]
                tweet_sentiment = sentiment[px]
                jaccard_score, _ = calculate_jaccard_score(
                    original_tweet=tweet,
                    target_string=selected_tweet,
                    sentiment_val=tweet_sentiment,
                    idx_start=np.argmax(outputs_start[px, :]),
                    idx_end=np.argmax(outputs_end[px, :]),
                    offsets=offsets[px]
                )
                jaccard_scores.append(jaccard_score)

            # Update running jaccard score and loss
            jaccards.update(np.mean(jaccard_scores), ids.size(0))
            losses.update(loss.item(), ids.size(0))
            # Print the running average loss and jaccard score
            tk0.set_postfix(loss=losses.avg, jaccard=jaccards.avg)
    
    print(f"Jaccard = {jaccards.avg}")
    return jaccards.avg

In [ ]:
def run(dfx, aug_df, fold, epochs=3):
    """
    Train model for a speciied fold
    """
    # Read training csv
#     dfx = pd.read_csv(TRAINING_FILE)

    # Set train validation set split
    df_train = dfx[dfx.kfold != fold].reset_index(drop=True)
    df_train = pd.concat([df_train, aug_df], axis=0)
    df_train = df_train[~df_train.is_incorrect]
    df_train = df_train.reset_index(drop=True)
    df_valid = dfx[dfx.kfold == fold].reset_index(drop=True)
    df_valid['selected_text'] = df_valid['answer']
    
    # Instantiate TweetDataset with training data
    train_dataset = TweetDataset(
        tweet=df_train.text.values,
        sentiment=df_train.sentiment.values,
        selected_text=df_train.selected_text.values
    )

    # Instantiate DataLoader with `train_dataset`
    # This is a generator that yields the dataset in batches
    train_data_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=TRAIN_BATCH_SIZE,
        num_workers=4
    )

    # Instantiate TweetDataset with validation data
    valid_dataset = TweetDataset(
        tweet=df_valid.text.values,
        sentiment=df_valid.sentiment.values,
        selected_text=df_valid.selected_text.values
    )

    # Instantiate DataLoader with `valid_dataset`
    valid_data_loader = torch.utils.data.DataLoader(
        valid_dataset,
        batch_size=VALID_BATCH_SIZE,
        num_workers=2
    )

    # Set device as `cuda` (GPU)
    device = torch.device("cuda")
    # Load pretrained RoBERTa
    model_config = transformers.RobertaConfig.from_pretrained(ROBERTA_PATH)
    # Output hidden states
    # This is important to set since we want to concatenate the hidden states from the last 2 BERT layers
    model_config.output_hidden_states = True
    # Instantiate our model with `model_config`
    model = TweetModel(conf=model_config)
    # Move the model to the GPU
    model.to(device)

    # Calculate the number of training steps
    num_train_steps = int(len(df_train) / TRAIN_BATCH_SIZE * EPOCHS)
    # Get the list of named parameters
    param_optimizer = list(model.named_parameters())
    # Specify parameters where weight decay shouldn't be applied
    no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
    no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight", "bert"]
    # Define two sets of parameters: those with weight decay, and those without
    optimizer_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.001},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
    ]
    # Instantiate AdamW optimizer with our two sets of parameters, and a learning rate of 3e-5
    optimizer = AdamW(optimizer_parameters, lr=3e-5)
    # Create a scheduler to set the learning rate at each training step
    # "Create a schedule with a learning rate that decreases linearly after linearly increasing during a warmup period." (https://pytorch.org/docs/stable/optim.html)
    # Since num_warmup_steps = 0, the learning rate starts at 3e-5, and then linearly decreases at each training step
    scheduler = get_linear_schedule_with_warmup(
        optimizer, 
        num_warmup_steps=0, 
        num_training_steps=num_train_steps
    )

    # Apply early stopping with patience of 2
    # This means to stop training new epochs when 2 rounds have passed without any improvement
    es = utils.EarlyStopping(patience=2, mode="max")
    print(f"Training is Starting for fold={fold}")
    
    score = 0
    # I'm training only for 3 epochs even though I specified 5!!!
    for epoch in range(epochs):
        train_fn(train_data_loader, model, optimizer, device, scheduler=scheduler)
        jaccard = eval_fn(valid_data_loader, model, device)
        print(f"Jaccard Score = {jaccard}")
        es(jaccard, model, model_path=f"model_{fold}.bin")
        score = max(score, jaccard)
        if es.early_stop:
            print("Early stopping")
            break
    return score

In [ ]:
from sklearn.model_selection import KFold

train = pd.read_csv(TRAINING_FILE)
train['answer'] = train['selected_text']
train['kfold'] = None
folds = 5
epochs = 3

mod_text = lambda row: " ".join(str(row.text).split()) # 2つスペースが入っていたりする
train['text'] = train.apply(mod_text, axis=1)
mod_text = lambda row: " ".join(str(row.selected_text).split()) # 2つスペースが入っていたりする
train['selected_text'] = train.apply(mod_text, axis=1)

is_incorrect_fn = lambda row: (" " + row.text + " ").find(" " + row.selected_text + " ") < 0
train['is_incorrect'] = train.apply(is_incorrect_fn, axis=1) # Trueが誤り。ややこしい。

# def mod_selected_text(row):
#   try:
#     text = row.text.split()
#     selected_text = row.selected_text.split()
#     ans = [None] * len(selected_text)
#     for i, s in enumerate(selected_text):
#       for j, t in enumerate(text):
#         if s == t:
#           ans[i] = j
#           break
#     for k, a in enumerate(ans):
#       if a is not None:
#         for kk in range(k):
#           ans[kk] = a-k+kk
#         for kk in range(1, len(ans)-k):
#           ans[k+kk] = a+kk
        
#         if max(ans) >= len(text):
#           break
#         te = [text[an] for an in ans]
#         if ' '.join(te) in ' '.join(text):
#           return ' '.join(te)
#     return row.selected_text
#   except:
#     print(row.text)
#     print(row.selected_text)
#     raise ZeroDivisionError

def mod_selected_text(row):
  text = row.text.split()
  selected_text = row.selected_text.split()
  for i, s in enumerate(selected_text):
    for t in text:
      if s in t and s!=t:
        selected_text[i] = t
        if ' '.join(selected_text) in ' '.join(text):
          return ' '.join(selected_text)
  return row.selected_text

train.loc[train.is_incorrect, 'selected_text'] = train[train.is_incorrect].apply(mod_selected_text, axis=1)

# neutralは９割方全文だしいでいいよね？
# train.loc[(train.is_incorrect) & (train.sentiment=='neutral'), 'selected_text'] = train.loc[(train.is_incorrect) & (train.sentiment=='neutral'), 'text']


train.loc[train.is_incorrect, 'is_incorrect'] = train[train.is_incorrect].apply(is_incorrect_fn, axis=1)
train = train.reset_index(drop=True)


skf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=42)
kf = KFold(n_splits = folds, shuffle = True)

for i, (train_index, test_index) in enumerate(skf.split(train, train.is_incorrect)):
# for i, (train_index, test_index) in enumerate(kf.split(train)):
    train.loc[test_index, 'kfold'] = i
train.head()

,textID,text,selected_text,sentiment,kfold,answer,is_incorrect
0,171b4de425,graduates college on saturday,graduates college on saturday,neutral,1,graduates college on saturday,False
1,ed37eaaf83,"thanks i have to finish schoolwork today, no r...",thanks,positive,1,thanks,False
2,3a2c407f74,how come when i straighten my hair it has to s...,how come when i straighten my hair it has to s...,neutral,2,how come when i straighten my hair it has to s...,False
3,cf966eb7b8,thanks Todd. Enjoyed reading your blog too - I...,Enjoyed,positive,4,Enjoyed,False
4,2a2b5d0558,"Oh my Lord, I have no idea if any of this ****...",****,negative,0,****,False


In [ ]:
# train.loc[(train.is_incorrect) & (train.sentiment=='neutral'), 'selected_text'] = train.loc[(train.is_incorrect) & (train.sentiment=='neutral'), 'text']
len(train[train.is_incorrect])

1176

In [ ]:
idx = 5687
print(train.iloc[idx].sentiment)
print(train.iloc[idx].text)
print(train.iloc[idx].selected_text)

positive
Thinking of ways to promote Indie Designers. Benefiting them and moi Any thoughts? Indies, what you do have to say?
Benefiting thoughts?


In [ ]:
for idx in train[train.is_incorrect].head(20).index:
  print(f"train.loc[{idx}, 'selected_text'] = \"\"")

train.loc[5518, 'selected_text'] = ""
train.loc[5544, 'selected_text'] = ""
train.loc[5681, 'selected_text'] = ""
train.loc[5687, 'selected_text'] = ""
train.loc[5700, 'selected_text'] = ""
train.loc[5706, 'selected_text'] = ""
train.loc[5713, 'selected_text'] = ""
train.loc[5728, 'selected_text'] = ""
train.loc[5740, 'selected_text'] = ""
train.loc[5755, 'selected_text'] = ""
train.loc[5767, 'selected_text'] = ""
train.loc[5829, 'selected_text'] = ""
train.loc[5836, 'selected_text'] = ""
train.loc[5896, 'selected_text'] = ""
train.loc[5913, 'selected_text'] = ""
train.loc[5924, 'selected_text'] = ""
train.loc[5965, 'selected_text'] = ""
train.loc[5981, 'selected_text'] = ""
train.loc[6078, 'selected_text'] = ""
train.loc[6092, 'selected_text'] = ""


In [ ]:
aug_df = pd.read_csv(os.path.join(DATA_PATH, 'pseudo_labels.csv'))
aug_df.drop([5578], inplace=True)
aug_df = aug_df.reset_index(drop=True) # まじここみすった
# aug_df.loc[519, 'selected_text'] = " can't keep eyes open....*shuts eyes*"
# aug_df.loc[1164, 'selected_text'] = ' cried so hard for a show..finally'
# aug_df.loc[1788, 'selected_text'] = " - arrggh kids that won't settle....need"
# aug_df.loc[2557, 'selected_text'] = ' tokio hotel-reden.and i must laugh.you'
# aug_df.loc[2604, 'selected_text'] = " i'm so very tired...and have insomnia."
# aug_df.loc[2675, 'selected_text'] = " can't wait to get out of here today...get"
# aug_df.loc[3096, 'selected_text'] = " feel like dressing up for the heck of it.&amp;"
# aug_df.loc[3742, 'selected_text'] = " that's why i need to be there...to represent the blackberries"
# aug_df.loc[3803, 'selected_text'] = " i can't stop crying......i"
# aug_df.loc[4447, 'selected_text'] = " what a riot..now"
# aug_df.loc[5303, 'selected_text'] = " tomorrow he'll gone to new home...i"
# aug_df.loc[5502, 'selected_text'] = " win something"
# aug_df.loc[5664, 'selected_text'] = " i don't have ur #...anymore, i think!!"
# aug_df.loc[6306, 'selected_text'] = " just finished the village. so in love,could"
# aug_df.loc[6847, 'selected_text'] = " why am i up...i"
# aug_df.loc[6920, 'selected_text'] = " won an international prize?and"
# aug_df.loc[7866, 'selected_text'] = " feel horrible...and"
# aug_df.loc[7976, 'selected_text'] = " ur so cute..i'm"
# aug_df.loc[8189, 'selected_text'] = " i'm just sooo in love....i think"
# aug_df.loc[8327, 'selected_text'] = " haha i forgot...it'll"
aug_df.drop([2169], inplace=True)
aug_df = aug_df.reset_index(drop=True)

mod_text = lambda row: " ".join(str(row.text).split()) # 2つスペースが入っていたりする
aug_df['text'] = aug_df.apply(mod_text, axis=1)
is_incorrect_fn = lambda row: (" " + row.text + " ").find(row.selected_text + " ") < 0
aug_df['is_incorrect'] = aug_df.apply(is_incorrect_fn, axis=1) # Trueが誤り。ややこしい。


# aug_df.loc[aug_df.is_incorrect, 'selected_text'] = aug_df[aug_df.is_incorrect].apply(mod_selected_text, axis=1)
is_incorrect_fn = lambda row: (" " + row.text + " ").find(row.selected_text + " ") < 0
aug_df.loc[aug_df.is_incorrect, 'is_incorrect'] = aug_df[aug_df.is_incorrect].apply(is_incorrect_fn, axis=1)

aug_df.head()

,sentiment,text,selected_text,is_incorrect
0,positive,so great to see oin &amp; cynthia. so happy. d...,so great to see oin &amp; cynthia. so happy.,False
1,positive,bought a new combo printer at costco today (of...,cheaper than ink 4 old one. #windows7 doesn't...,False
2,positive,freedom,freedom,False
3,negative,its funny. i'm kinda sad on my last day,sad,False
4,positive,@_bella_cullen13 @rosaliehcullen @esme_cullen2...,girls love,False


In [ ]:
scores = []
folds = 5
epochs = 5

for i in range(folds):
  scores.append(run(train, aug_df, fold=i, epochs=epochs))

Training is Starting for fold=0



Jaccard = 0.7059872923594871
Jaccard Score = 0.7059872923594871
Validation score improved (-inf --> 0.7059872923594871). Saving model!



Jaccard = 0.7112119593400059
Jaccard Score = 0.7112119593400059
Validation score improved (0.7059872923594871 --> 0.7112119593400059). Saving model!



Jaccard = 0.710174972761026
Jaccard Score = 0.710174972761026
EarlyStopping counter: 1 out of 2



Jaccard = 0.710174972761026
Jaccard Score = 0.710174972761026
EarlyStopping counter: 2 out of 2
Early stopping
Training is Starting for fold=1


...


Jaccard = 0.7119368214156931
Jaccard Score = 0.7119368214156931
Validation score improved (0.7079558118272727 --> 0.7119368214156931). Saving model!



Jaccard = 0.711482715275929
Jaccard Score = 0.711482715275929
EarlyStopping counter: 1 out of 2



Jaccard = 0.711482715275929
Jaccard Score = 0.711482715275929
EarlyStopping counter: 2 out of 2
Early stopping
Training is Starting for fold=4



Jaccard = 0.705114080303944
Jaccard Score = 0.705114080303944
Validation score improved (-inf --> 0.705114080303944). Saving model!



Jaccard = 0.7098621765678351
Jaccard Score = 0.7098621765678351
Validation score improved (0.705114080303944 --> 0.7098621765678351). Saving model!


In [ ]:
# base kernel
# Jaccard Scores:  [0.6992109016492154, 0.6957094991568392, 0.7057985564815754, 0.7056903926604363, 0.7088782537360205]
# Jaccard Scores mean:  0.7030575207368174

# aug + 機械的に汚いデータを修正した epoch5
# [0.7094770154943421, 0.7080641044949311, 0.7153339471644156, 0.7086386144581275, 0.7144430211066117]
# 0.7111913405436856

# aug + 機械的に汚いデータを修正した
# Jaccard Scores:  [0.7084673182098051, 0.7040050862213606, 0.7148638406623687, 0.7107141519230824, 0.7114956536754835]
# Jaccard Scores mean:  0.7099092101384201

# aug + 綺麗なデータのみ epoch5
# [0.7096246869916724, 0.7091477460211161, 0.7187250143345432, 0.7031187460817482, 0.7119063699121203]
# 0.71050451266824

# aug lbは変わらず
# Jaccard Scores:  [0.7122321865332638, 0.7019863400435968, 0.7107366422164924, 0.7165641217070267, 0.7053660974514369]
# Jaccard Scores mean:  0.7093770775903633

# kerasのやつ再現
# sum([0.6990500009626294, 0.6950044181706834, 0.709157277676115, 0.7010584909594423, 0.7070522393215964]) / 5
# 0.7022644854180933

# [Commented] RoBERTa training with PyTorch : 7fold
# Jaccard Scores:  [0.6955231231183197, 0.706686294644901, 0.7103266720898102, 0.690334342481451, 0.7107565154784258, 0.7049763523167129, 0.7099995103355424]
# Jaccard Scores mean:  0.7040861157807374

# conv追加
# Jaccard Scores:  [0.699477504729838, 0.698912368337463, 0.7068319181353215, 0.7058647760547342, 0.7053416922691689]
# Jaccard Scores mean:  0.7032856519053052

# conv層深く
# Jaccard Scores:  [0.7018605108997668, 0.7002823487753598, 0.7074263341072304, 0.7088355439914501, 0.7067711277065578]
# Jaccard Scores mean:  0.7050351730960729

# conv dropout追加したり
# Jaccard Scores:  [0.7000203645975672, 0.6998912734318095, 0.70566229228583, 0.70709872108416, 0.7012402306969029]
# Jaccard Scores mean:  0.7027825764192539

# conv lossを2回計算、bn->relu->conv
# Jaccard Scores:  [0.6991273629135035, 0.6952953010856404, 0.7070218519800546, 0.7056670835366958, 0.7093855179784171]
# Jaccard Scores mean:  0.7032994234988623

print('Jaccard Scores: ', scores)
print('Jaccard Scores mean: ', str(sum(scores) / len(scores)))

Jaccard Scores:  [0.6992109016492154, 0.6957094991568392, 0.7057985564815754, 0.7056903926604363, 0.7088782537360205]
Jaccard Scores mean:  0.7030575207368174


In [ ]:
run(train, fold=0, epochs=5)

Training is Starting for fold=0



Jaccard = 0.6905522084989205
Jaccard Score = 0.6905522084989205
Validation score improved (-inf --> 0.6905522084989205). Saving model!



Jaccard = 0.6964189058198782
Jaccard Score = 0.6964189058198782
Validation score improved (0.6905522084989205 --> 0.6964189058198782). Saving model!



Jaccard = 0.6990121297194826
Jaccard Score = 0.6990121297194826
Validation score improved (0.6964189058198782 --> 0.6990121297194826). Saving model!



Jaccard = 0.6987479950911492
Jaccard Score = 0.6987479950911492
EarlyStopping counter: 1 out of 2



Jaccard = 0.6986223854512591
Jaccard Score = 0.6986223854512591
EarlyStopping counter: 2 out of 2
Early stopping


0.6990121297194826

# Do the evaluation on test data

In [ ]:
df_test = pd.read_csv("./data/test.csv")
df_test.loc[:, "selected_text"] = df_test.text.values

In [ ]:
device = torch.device("cuda")
model_config = transformers.RobertaConfig.from_pretrained(ROBERTA_PATH)
model_config.output_hidden_states = True

In [ ]:
models = []

for i in range(folds):
  model = TweetModel(conf=model_config)
  model.to(device)
  model.load_state_dict(torch.load(f"model_{i}.bin"))
  model.eval()
  models.append(model)

In [ ]:
final_output = []

# Instantiate TweetDataset with the test data
test_dataset = TweetDataset(
        tweet=df_test.text.values,
        sentiment=df_test.sentiment.values,
        selected_text=df_test.selected_text.values
)

# Instantiate DataLoader with `test_dataset`
data_loader = torch.utils.data.DataLoader(
    test_dataset,
    shuffle=False,
    batch_size=VALID_BATCH_SIZE,
    num_workers=1
)

# Turn of gradient calculations
with torch.no_grad():
    tk0 = tqdm(data_loader, total=len(data_loader))
    # Predict the span containing the sentiment for each batch
    for bi, d in enumerate(tk0):
        ids = d["ids"]
        token_type_ids = d["token_type_ids"]
        mask = d["mask"]
        sentiment = d["sentiment"]
        orig_selected = d["orig_selected"]
        orig_tweet = d["orig_tweet"]
        targets_start = d["targets_start"]
        targets_end = d["targets_end"]
        offsets = d["offsets"].numpy()

        ids = ids.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        targets_start = targets_start.to(device, dtype=torch.long)
        targets_end = targets_end.to(device, dtype=torch.long)


        avg_outputs_start = torch.zeros([len(orig_tweet), MAX_LEN], device=device)
        avg_outputs_end = torch.zeros([len(orig_tweet), MAX_LEN], device=device)
        for model in models:
            outputs_start, outputs_end = model(ids, mask, token_type_ids)
            avg_outputs_start += outputs_start / len(models)
            avg_outputs_end += outputs_end / len(models)

        outputs_start = torch.softmax(avg_outputs_start, dim=1).cpu().detach().numpy()
        outputs_end = torch.softmax(avg_outputs_end, dim=1).cpu().detach().numpy()

        # Convert the start and end scores to actual predicted spans (in string form)
        for px, tweet in enumerate(orig_tweet):
            selected_tweet = orig_selected[px]
            tweet_sentiment = sentiment[px]
            _, output_sentence = calculate_jaccard_score(
                original_tweet=tweet,
                target_string=selected_tweet,
                sentiment_val=tweet_sentiment,
                idx_start=np.argmax(outputs_start[px, :]),
                idx_end=np.argmax(outputs_end[px, :]),
                offsets=offsets[px]
            )
            final_output.append(output_sentence)

In [ ]:
def post_process(selected):
    return " ".join(set(selected.lower().split()))

In [ ]:
sample = pd.read_csv("./data/sample_submission.csv")
sample.loc[:, 'selected_text'] = final_output
sample.selected_text = sample.selected_text.map(post_process)
sample.to_csv("submission.csv", index=False)

In [ ]:
sample.head()

,textID,selected_text
0,f87dea47db,session the of day last
1,96d74cb729,exciting
2,eee518ae67,shame!
3,01082688c6,happy
4,33987a8ee5,it!! i like
